In [1]:
!pip install optuna

You should consider upgrading via the 'c:\users\llewyn\code\comp470277032021\machine_learning\scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import tensorflow as tf
%load_ext tensorboard
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Model
import datetime
import optuna
import pandas as pd
import numpy as np

In [2]:
def get_diabetes(batch_size):
    diabeetus = pd.read_csv('pima_indians_diabetes.csv')
    diabeetusY = diabeetus.pop('9')
    diabeetusY[diabeetusY == 'pos'] = 1
    diabeetusY[diabeetusY == 'neg'] = 0
    dataset = tf.data.Dataset.from_tensor_slices((diabeetus.values.astype(np.float32), diabeetusY.values.astype(int))).shuffle(10000).batch(batch_size)
    validation_dataset = dataset.take(int(diabeetus.shape[0]*0.20)) 
    train_dataset = dataset.skip(int(diabeetus.shape[0]*0.20))
    return train_dataset, validation_dataset

In [6]:
#stolen from https://github.com/optuna/optuna/blob/master/examples/tensorflow/tensorflow_eager_simple.py
CLASSES = 2

def create_model(trial):
    # We optimize the numbers of layers, their units and weight decay parameter.
    n_layers = trial.suggest_int("n_layers", 1, 2)
#     weight_decay = 0
    act_list = ["elu", "relu"]
    act = trial.suggest_categorical("activation_function", act_list)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for i in range(n_layers):
        num_hidden = trial.suggest_int("n_units_l{}".format(i), 5, 100)
        model.add(
            tf.keras.layers.Dense(
                num_hidden,
                activation=act,
#                 kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
            )
        )
    model.add(
        tf.keras.layers.Dense(1, activation='sigmoid')
    )
#     model.add(tf.keras.layers.Activation(tf.math.round))
    return model


def create_optimizer(trial):
    # We optimize the choice of optimizers as well as their parameters.
    kwargs = {}
    optimizer_selected = "Adam"
#     optimizer_options = ["RMSprop", "Adam", "SGD"]
#     optimizer_selected = trial.suggest_categorical("optimizer", optimizer_options)
    
#     if optimizer_selected == "RMSprop":
#         kwargs["learning_rate"] = trial.suggest_float(
#             "rmsprop_learning_rate", 1e-5, 1e-1, log=True
#         )
#         kwargs["decay"] = trial.suggest_float("rmsprop_decay", 0.85, 0.99)
#         kwargs["momentum"] = trial.suggest_float("rmsprop_momentum", 1e-5, 1e-1, log=True)
#     elif optimizer_selected == "Adam":
#         kwargs["learning_rate"] = trial.suggest_float("adam_learning_rate", 1e-5, 1e-1, log=True)
#     elif optimizer_selected == "SGD":
#         kwargs["learning_rate"] = trial.suggest_float(
#             "sgd_opt_learning_rate", 1e-5, 1e-1, log=True
#         )
#         kwargs["momentum"] = trial.suggest_float("sgd_opt_momentum", 1e-5, 1e-1, log=True)
    
    optimizer = getattr(tf.optimizers, optimizer_selected)(**kwargs)
    return optimizer


def learn(model, optimizer, dataset, mode="eval"):
    accuracy = tf.metrics.Accuracy("accuracy", dtype=tf.float32)

    for batch, (images, labels) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = model(images, training=(mode == "train"))
            loss_value = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
            )
            if mode == "eval":
                accuracy(
                    tf.cast(tf.math.round(logits), tf.int64), tf.cast(labels, tf.int64)
                )
            else:
                grads = tape.gradient(loss_value, model.variables)
                optimizer.apply_gradients(zip(grads, model.variables))

    if mode == "eval":
        return accuracy


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    # Get diabetes data.
    bs = 64
    train_ds, valid_ds = get_diabetes(bs)

    # Build model and optimizer.
    model = create_model(trial)
    optimizer = create_optimizer(trial)

    # Training and validating cycle.
    with tf.device("/gpu:0"):
        for _ in range(2000):
            learn(model, optimizer, train_ds, "train")

        accuracy = learn(model, optimizer, valid_ds, "eval")

    # Return last validation accuracy.
    return accuracy.result()

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-05-20 13:28:40,283] Finished trial#0 with value: 0.421875 with parameters: {'n_layers': 1, 'activation_function': 'relu', 'n_units_l0': 71}. Best is trial#0 with value: 0.421875.
[I 2021-05-20 13:28:57,404] Finished trial#1 with value: 0.4518229067325592 with parameters: {'n_layers': 2, 'activation_function': 'relu', 'n_units_l0': 66, 'n_units_l1': 45}. Best is trial#1 with value: 0.4518229067325592.
[I 2021-05-20 13:29:14,691] Finished trial#2 with value: 0.3489583432674408 with parameters: {'n_layers': 2, 'activation_function': 'relu', 'n_units_l0': 92, 'n_units_l1': 61}. Best is trial#1 with value: 0.4518229067325592.
[I 2021-05-20 13:29:32,282] Finished trial#3 with value: 0.6510416865348816 with parameters: {'n_layers': 1, 'activation_function': 'relu', 'n_units_l0': 36}. Best is trial#3 with value: 0.6510416865348816.
[I 2021-05-20 13:29:49,513] Finished trial#4 with value: 0.6458333134651184 with parameters: {'n_layers': 1, 'activation_function': 'elu', 'n_units_l0': 100

In [77]:
def create_model():
    # We optimize the numbers of layers, their units and weight decay parameter.
    n_layers = 1
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for i in range(n_layers):
        num_hidden = 5
        model.add(
            tf.keras.layers.Dense(
                num_hidden,
                activation='tanh',
                kernel_initializer=tf.keras.initializers.RandomUniform(minval=-0.5, maxval=0.5),
                use_bias=False

#                 kernel_regularizer=tf.keras.regularizers.l2(weight_decay),
            )
        )
    model.add(
        tf.keras.layers.Dense(1, activation='sigmoid')
    )
#     model.add(tf.keras.layers.Activation(tf.math.round))
    return model




def learn(model, optimizer, dataset, mode="eval"):
    accuracy = tf.metrics.Accuracy("accuracy", dtype=tf.float32)

    for batch, (images, labels) in enumerate(dataset):
        with tf.GradientTape() as tape:
            logits = model(images, training=(mode == "train"))
            loss_value = tf.keras.losses.Hinge(labels, logits)

            if mode == "eval":
                accuracy(
                    tf.cast(tf.math.round(logits), tf.float32), tf.cast(labels, tf.float32)
                )
            else:
                grads = tape.gradient(loss_value, model.variables)
                optimizer.apply_gradients(zip(grads, model.variables))

    if mode == "eval":
        return accuracy

def objective():
    # Get diabetes data.
    bs = 768
    train_ds, valid_ds = get_diabetes(bs)

    # Build model and optimizer.
    model = create_model()
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.15, momentum=0.9)

    # Training and validating cycle.
    for _ in range(30):
        learn(model, optimizer, train_ds, "train")

    accuracy = learn(model, optimizer, valid_ds, "eval")

    # Return last validation accuracy.
    return accuracy.result()

In [78]:
some_list = []
for i in range(50):
    some_list.append(objective().numpy())
np.array(some_list).max()


TypeError: Cannot convert 'auto' to EagerTensor of dtype int32

In [66]:
some_list

[0.35416666,
 0.6510417,
 0.64713544,
 0.63671875,
 0.40104166,
 0.49479166,
 0.55078125,
 0.5494792,
 0.66015625,
 0.64973956,
 0.3671875,
 0.50651044,
 0.5859375,
 0.3502604,
 0.41796875,
 0.6510417,
 0.5182292,
 0.48046875,
 0.390625,
 0.6380208]